In [78]:
%run func.ipynb
import mahotas
from pyfeats import *
import pandas as pd
import numpy as np
import time

from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('sidikjari.csv')

In [5]:
df.head()

,Mean,Variance,Median,Mode,Skewness,Kurtosis,Energy,EntropyFOS,MinimalGrayLevel,MaximalGrayLevel,...,SumAverage,SumVariance,SumEntropy,EntropyGLCM,DifferenceVariance,DifferenceEntropy,Information1,Information2,MaximalCorrelationCoefficient,Target
0,152.446120,10519.917129,185.0,255.0,-0.448559,1.541112,0.112145,3.886009,0.0,255.0,...,378.961310,17978.482526,7.026904,9.521569,0.000325,6.212765,-0.334840,0.988978,2.093801,1
1,164.632274,10867.792041,220.0,255.0,-0.632901,1.674024,0.185741,3.347604,0.0,255.0,...,405.111014,18233.198107,5.703622,7.471341,0.000813,5.052027,-0.393346,0.986913,2.220650,2
2,127.570227,10963.166831,133.0,0.0,-0.030917,1.332301,0.103204,3.948212,0.0,255.0,...,345.618017,20659.887617,7.440459,10.069340,0.000228,6.597364,-0.342323,0.992020,2.247461,3
3,133.592944,10393.465348,151.0,255.0,-0.142980,1.395373,0.087368,4.130406,0.0,255.0,...,344.503904,19971.467290,7.523537,10.391891,0.000213,6.600903,-0.348631,0.993648,2.251734,4
4,154.199530,10379.337585,189.0,255.0,-0.482152,1.580665,0.107202,3.939134,0.0,255.0,...,376.630395,17469.220331,7.182256,9.766996,0.000279,6.373085,-0.320211,0.987256,2.058490,5


In [6]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

len_row, len_col = X.shape
print('Row: ',len_row)

X_col = X.columns.tolist()

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the dataset
scaler.fit(X)

# Transform the dataset using the scaler
X_std = scaler.transform(X)

X_std = pd.DataFrame(data=X_std, columns=X_col)

Row:  100


In [7]:
X_std.head()

,Mean,Variance,Median,Mode,Skewness,Kurtosis,Energy,EntropyFOS,MinimalGrayLevel,MaximalGrayLevel,...,InverseDifferenceMoment,SumAverage,SumVariance,SumEntropy,EntropyGLCM,DifferenceVariance,DifferenceEntropy,Information1,Information2,MaximalCorrelationCoefficient
0,0.379679,-0.685698,0.254859,0.546536,-0.441390,0.209352,-0.819742,0.831827,0.0,0.0,...,-0.679274,-0.064086,-0.528244,0.645357,0.679256,-0.701367,0.735735,0.819996,0.211461,-1.096791
1,0.938440,-0.335966,0.885253,0.546536,-0.947796,0.727815,0.440025,-0.341317,0.0,0.0,...,0.617165,0.681464,-0.453942,-0.561607,-0.552122,0.468258,-0.515583,-0.215012,-0.061895,-0.822404
2,-0.760933,-0.240082,-0.681726,-1.829707,0.705918,-0.605176,-0.972783,0.967361,0.0,0.0,...,-1.067036,-1.014730,0.253935,1.022561,1.008250,-0.932798,1.150346,0.687630,0.614001,-0.764409
3,-0.484779,-0.812825,-0.357523,0.546536,0.398070,-0.359147,-1.243855,1.364347,0.0,0.0,...,-1.130466,-1.046495,0.053120,1.098335,1.201976,-0.967683,1.154161,0.576036,0.829335,-0.755166
4,0.460077,-0.827029,0.326904,0.546536,-0.533674,0.363637,-0.904352,0.947581,0.0,0.0,...,-0.855564,-0.130542,-0.676798,0.787054,0.826661,-0.809995,0.908565,1.078803,-0.016456,-1.173172


In [76]:
# # without feature selection
# X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.3, random_state = 42)
# start_time = time.time()
# # Train a random forest model on the training data
# model = RandomForestClassifier(random_state = 42)
# model.fit(X_train, y_train)
# end_time = time.time()

# # Evaluate the model on the testing data
# y_pred = model.predict(X_test)
# acc = accuracy_score(y_test, y_pred)

# print(f'Accuracy = {acc}')
# print(f"Elapsed time: {end_time - start_time} seconds")
# print()

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size = 0.3, random_state = 42)

# Define the range of values for k
k_values = range(1, len_col+1)
# Initialize dictionaries to store results
accs = {}
selected_each_k = {}
timer = {}

for k in k_values:
    scores, selected, scores_ith, score_df, relevancy, redundancy = min_redun_max_relev(X, y, k)
    selected_each_k[f'k = {k}'] = selected
    # Remove all columns from X that are not selected
    X_selected = X_train[selected]
    
    start_time = time.time()
    # Train a random forest model on the training data
    model = RandomForestClassifier(random_state = 42)
    model.fit(X_selected, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_selected_test = X_test[selected]
    y_pred = model.predict(X_selected_test)
    acc = accuracy_score(y_test, y_pred)
    
    times = end_time - start_time
    
    timer[f'k = {k}'] = times
    accs[f'k = {k}'] = acc

In [79]:
X_selected.head()

,Contrast,Correlation,EntropyFOS,90Percentile,Energy,DifferenceEntropy,InverseDifferenceMoment,Information1,EntropyGLCM,MaximalCorrelationCoefficient,...,Mean,Skewness,10Percentile,HistogramWidth,Median,SumVariance,75Percentile,Mode,MinimalGrayLevel,MaximalGrayLevel
11,-0.671100,0.741316,-1.099522,0.100504,0.914762,-0.959488,1.017969,-0.325996,-1.065551,0.306908,...,0.203484,-0.134390,-0.274611,0.278762,0.443978,0.160263,0.321304,0.546536,0.0,0.0
47,1.100326,-0.657052,0.849506,0.100504,-0.863587,1.122629,-1.118979,0.379871,1.067896,-0.155369,...,-1.103701,0.991399,-0.274611,0.278762,-1.059963,0.439758,-0.504442,-1.829707,0.0,0.0
85,-0.164914,0.000283,0.062704,0.100504,-0.253781,-0.143782,0.187460,-0.532143,-0.094393,0.059757,...,-0.093223,0.042887,-0.274611,0.278762,-0.051332,-0.212799,0.321304,0.546536,0.0,0.0
28,0.120261,0.441890,-0.243547,0.100504,-0.019385,-0.424373,0.544269,-1.095114,-0.483042,1.319734,...,-0.460431,0.471927,-0.274611,0.278762,-0.285478,0.941531,0.321304,0.546536,0.0,0.0
93,-1.645430,0.631229,-1.890064,0.100504,2.191837,-2.125744,2.006416,-1.450628,-1.981955,0.207191,...,1.457296,-1.522656,-0.274611,0.278762,1.515647,-1.730964,0.321304,0.546536,0.0,0.0


In [80]:
# Choose the best k based on the validation accuracies

print(f"MSE using all columns= {accs[f'k = {len_col}']}")
print(f"Elapsed Time = {timer[f'k = {len_col}']}")
print()
best_k = max(accs, key=accs.get)
print(f"Best {best_k} with Accuracy = {accs[best_k]}")
print(f"Elapsed Time = {timer[best_k]}")
print()
print(f"Accuracy using 3 best columns= {accs[f'k = 3']}")
print(f"Elapsed Time = {timer[f'k = 3']}")
print()
print(f"Accuracy using 10 best columns= {accs[f'k = 10']}")
print(f"Elapsed Time = {timer[f'k = 10']}")
print()

MSE using all columns= 0.3333333333333333
Elapsed Time = 0.20009303092956543

Best k = 6 with Accuracy = 0.43333333333333335
Elapsed Time = 0.19979238510131836

Accuracy using 3 best columns= 0.3
Elapsed Time = 0.19463801383972168

Accuracy using 10 best columns= 0.43333333333333335
Elapsed Time = 0.28789329528808594



In [87]:
worsts = [['Mode','MinimalGrayLevel','MaximalGrayLevel'],['Mean','Skewness','10Percentile','HistogramWidth','Median','SumVariance','75Percentile','Mode','MinimalGrayLevel','MaximalGrayLevel']]

for worst in worsts:
    # USING 3 Worst columns & USING 10 Worst columns
    X_select = X_selected[worst]

    start_time = time.time()
    # Train a random forest model on the training data
    model = RandomForestClassifier(random_state = 42)
    model.fit(X_select, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_selected_test = X_test[worst]
    y_pred = model.predict(X_selected_test)
    acc = accuracy_score(y_test, y_pred)

    times = end_time - start_time

    print(f"Accuracy using {len(worst)} worst columns= {acc}")
    print(f"Elapsed Time = {times}")
    print()

Accuracy using 3 worst columns= 0.16666666666666666
Elapsed Time = 0.19145941734313965

Accuracy using 10 worst columns= 0.23333333333333334
Elapsed Time = 0.19192886352539062



In [81]:
print(selected_each_k[best_k])

['Contrast', 'Correlation', 'EntropyFOS', '90Percentile', 'Energy', 'DifferenceEntropy']


In [91]:
pd.DataFrame(scores, columns=['mRMR', 'Highest_score_each_iteration']).head()

,mRMR,Highest_score_each_iteration
0,Contrast,1.519045e+06
1,Correlation,1.371383e+01
2,EntropyFOS,1.409696e+01
3,90Percentile,1.772721e+01
4,Energy,1.210128e+01


In [82]:
# Find the maximum number of values across all keys
max_values = max([len(val) if isinstance(val, list) else 1 for val in selected_each_k.values()])

# Iterate through the keys and add "n/a" values as necessary
for key, val in selected_each_k.items():
    # Check if the value is a string and split by comma if necessary
    if isinstance(val, str):
        values = val.split(',')
    else:
        values = val
        
    # If the number of values is less than the maximum, add "n/a" values
    num_values = len(values)
    if num_values < max_values:
        diff = max_values - num_values
        values += ['-'] * diff
    
    # Join the values with commas and update the dictionary
    selected_each_k[key] = values
    
selected_each_k_df = pd.DataFrame(selected_each_k)

In [83]:
selected_each_k_df.iloc[:, :100].head(100)

,k = 1,k = 2,k = 3,k = 4,k = 5,k = 6,k = 7,k = 8,k = 9,k = 10,...,k = 21,k = 22,k = 23,k = 24,k = 25,k = 26,k = 27,k = 28,k = 29,k = 30
0,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,...,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast,Contrast
1,-,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,...,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation,Correlation
2,-,-,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,...,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS,EntropyFOS
3,-,-,-,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,...,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile,90Percentile
4,-,-,-,-,Energy,Energy,Energy,Energy,Energy,Energy,...,Energy,Energy,Energy,Energy,Energy,Energy,Energy,Energy,Energy,Energy
5,-,-,-,-,-,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,...,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy,DifferenceEntropy
6,-,-,-,-,-,-,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,...,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment,InverseDifferenceMoment
7,-,-,-,-,-,-,-,Information1,Information1,Information1,...,Information1,Information1,Information1,Information1,Information1,Information1,Information1,Information1,Information1,Information1
8,-,-,-,-,-,-,-,-,EntropyGLCM,EntropyGLCM,...,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM,EntropyGLCM
9,-,-,-,-,-,-,-,-,-,MaximalCorrelationCoefficient,...,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient,MaximalCorrelationCoefficient


In [84]:
for i in range(len(scores_ith)):
    combines = {'Relevancy':relevancy[i], 'Redundancy':redundancy[i],'MRMR':scores_ith[i]}
    scored_ith = pd.DataFrame(combines)
    scored_ith = scored_ith.sort_values('MRMR', ascending=False)
    print('ITERASI KE ', i+1)
    print(scored_ith)
    print()
    print()

ITERASI KE  1
                               Relevancy  Redundancy          MRMR
Contrast                       15.190449     0.00001  1.519045e+06
Correlation                    11.054915     0.00001  1.105491e+06
DifferenceEntropy               6.557856     0.00001  6.557856e+05
EntropyFOS                      6.535353     0.00001  6.535353e+05
Energy                          6.243083     0.00001  6.243083e+05
InverseDifferenceMoment         6.053728     0.00001  6.053728e+05
SumEntropy                      5.171988     0.00001  5.171988e+05
EntropyGLCM                     5.138155     0.00001  5.138155e+05
Information1                    4.783042     0.00001  4.783042e+05
DifferenceVariance              4.693318     0.00001  4.693318e+05
ASM                             4.454130     0.00001  4.454130e+05
SumAverage                      3.090602     0.00001  3.090602e+05
SumOfSquaresVariance            3.052085     0.00001  3.052085e+05
90Percentile                    2.020619     0.0